In [4]:
import torch
from torch import nn
import torchvision

import matplotlib.pyplot as plt
from torchvision import transforms
from torchinfo import summary

from pathlib import Path

from going_modular.going_modular import engine, data_setup

## Device agnostic code

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
## Path

dataset_path = Path("/datasets/pizza_steak_sushi")
train_dir = dataset_path / "train"
test_dir = dataset_path / "test"

## Transform

data_transform = transforms.Compose([
    
])

train_dataloader, test_dataloader, classes = data_setup.create_dataloaders(train_dir=train_dir,
                                                                           test_dir=test_dir,
                                                                           )